In [1]:
import copy
import json
import requests
import numpy as np

url = "https://search-aiindex-76cl5n3qdcbspk7eztijpvdq6i.us-east-2.es.amazonaws.com/arxiv2/documents/_msearch?"

#Find arXiv papers, which contain "Ukraine"
data_template = [
  {
    "preference": "SearchResult"
  },
  {
    "query": {
      "bool": {
        "must": [
          {
             "bool": {
                    "should": [
                      {
                        "multi_match": {
                          "query": "Ukraine",
                          "fields": [
                            "content"
                          ],
                          "type": "phrase_prefix",
                          "operator": "or",
                        #  "fuzziness": 0,
                        }
                      }
                     
                    ],
                    "minimum_should_match": "1"
                  }},
              {"bool": {"should": [],  "minimum_should_match": "1"},
          }
        ]
      }
    },
    "size": 8,
    "_source": {
      "includes": [
        "*"
      ],
      "excludes": []
    },
    "from": 0
  }
]
headers = {"Content-type": "application/x-ndjson"}

all_hits = []


In [ ]:
'''cats_crit = []

for cat in relevant_categories:
    cats_crit.append( {
                        "multi_match": {
                          "query": cat,
                          "fields": [
                            "category"
                          ],
                          "type": "phrase_prefix",
                          "operator": "or"
                        }
                      })
data_template[1]['query']['bool']['must'][1]['bool']['should'] = cats_crit'''

In [2]:
#Adding relevant categories
relevant_categories = ["cs.LG","cs.CV", "cs.CL", "cs.RO", "stat.ML", "cs.AI", "cs.NE"]



In [26]:
#Parsing paper content and cropping paper name and authors

def get_title_and_authors(content):
    text = content['_source']['content']
    abstract_pos = []
    for abs_names in ['abstract', 'summary', 'keyword', "abst ract"]:
        pos = text.lower().find(abs_names)
        if pos > 0:
            abstract_pos.append(pos)
    if len(abstract_pos) > 0:
        abstract_pos = min(np.array(abstract_pos))
        title_and_authors = text[:abstract_pos]
    else:
        title_and_authors = ""
    return title_and_authors.strip()

def country_in_abstract(content, country = []):
    title_and_authors = get_title_and_authors(content)
    found = False
    for c in country:
        if c in title_and_authors:
            found = True
    return found, title_and_authors

def is_relevant_category(res, rel_cats):
    paper_cats = res['_source']['categories']
    cats_intersect =  set(paper_cats).intersection(set(relevant_categories))
    #print (cats_intersect)
    return len(cats_intersect) > 0

In [4]:
data_template

[{'preference': 'SearchResult'},
 {'query': {'bool': {'must': [{'bool': {'should': [{'multi_match': {'query': 'Ukraine',
          'fields': ['content'],
          'type': 'phrase_prefix',
          'operator': 'or'}}],
       'minimum_should_match': '1'}},
     {'bool': {'should': [], 'minimum_should_match': '1'}}]}},
  'size': 8,
  '_source': {'includes': ['*'], 'excludes': []},
  'from': 0}]

In [19]:
#Searching and storing
mx = 1
mx_id = 0
from time import sleep
all_hits = []
while mx_id < mx:
    data_to_post = copy.deepcopy(data_template)
    data_to_post[1]["from"] = mx_id
    data_to_post = '\n'.join(json.dumps(d) for d in data_to_post) + "\n"
    r = requests.post(url, headers=headers, data=data_to_post)
    info = json.loads(r.text)
    mx = info['responses'][0]['hits']["total"]['value']
    hits = info['responses'][0]['hits']['hits']
    all_hits += hits
    mx_id += len(hits)
    sleep (0.5)
    print(mx_id, mx)

print(len(all_hits))
with open("hits.json", "w") as f:
    json.dump(all_hits, f)


8 277
16 277
24 277
32 277
40 277
48 277
56 277
64 277
72 277
80 277
88 277
96 277
104 277
112 277
120 277
128 277
136 277
144 277
152 277
160 277
168 277
176 277
184 277
192 277
200 277
208 277
216 277
224 277
232 277
240 277
248 277
256 277
264 277
272 277
277 277
277


In [27]:
Ukrainian_papers = []
for res in all_hits:
    if not is_relevant_category(res, relevant_categories):
        continue
    good, title_and_authors = country_in_abstract(res, ['Ukrain', 'Lviv', "Kyiv", 'Kiev', 'Kharkiv', 'Odessa'])
    if good:
        Ukrainian_papers.append(title_and_authors)
        #print ("****")
        #print (TaA)
        #print (res['_source']['categories'])
print ("Found", len(Ukrainian_papers), "papers with Ukraine in affiliation")
with open("Ukr_papers_names_and_abstracts.json", "w") as f:
    json.dump(all_hits, f)

Found 97 papers with Ukraine in affiliation


In [28]:
for p in Ukrainian_papers:
    print (p)
    print ('*****')

Microsoft Word - IRFPatRec.doc


PATTERN RECOGNITION USING INVERSE RESONANCE FILTRATION 
 

Olga Sofina1, Yuriy Bunyak2, Roman Kvetnyy3 
 

1 Vinnitsa National Technical University. 95 Khmelnitske sh. Vinnitsa 21021 Ukraine. olya_bunyak@mail.ru  
2 InnoVinn Inc. 14 Kievskaya str. Vinnitsa 21001 Ukraineinn. yuri.bunyak@innovinn.com 
3 Vinnitsa National Technical University. 95 Khmelnitske sh. Vinnitsa 21021 Ukraine. rkvetny@mail.ru
*****
Microsoft Word - newart5.doc


  

Forecasting of Events by Tweet Data Mining  
 

Bohdan Pavlyshenko  
 

Ivan Franko Lviv National University,Ukraine,  b.pavlyshenko@gmail.com
*****
Paper Title (use style: paper title)


 

XXX-X-XXXX-XXXX-X/XX/$XX.00 ©20XX IEEE 

a) 

b) 
Fig. 1. Example of the original image for the medieval graffito #3 (c. 
1022) in St. Sophia Cathedral of Kyiv (Ukraine) (a) and preprocessed 
glyphs (b) which are translated to "in 6530 (c. 1022) O Lord, save" [3] 

   

Capsule Deep Neural Network for Recognition of 
Historical Gra